# Rover Model Explanation and Preliminaries

This notebook covers the setup of a lane-following rover fault model for understanding the effects of faults in AI-driven systems. This model uses the fmdtools simulation toolkit to simulate the nominal and faulty behaviors of the rover over a set of fault scenarios and classify/assess risk.

In [1]:
import fmdtools.analyze as an
import fmdtools.sim.propagate as prop
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

This model is defined in `rover_model.py`.
- `Rover` is the class defining the model, 
- `gen_params` is a function used to generate model parameters (given design variables), and 
- `plot_map` is used to visualize the trajectory of the rover over time

In [2]:
from rover_model import Rover, gen_params, plot_map, plot_trajectories

ImportError: cannot import name 'gen_params' from 'rover_model' (C:\Users\dhulse\Documents\GitHub\fmdtools\examples\rover\rover_model.py)

## Model Setup

In [3]:
mdl = Rover()

In [4]:
#%matplotlib qt
#an.graph.set_pos(mdl, gtype='fxnflowgraph')

The functions and flows of the model are shown below, and were taken from the design of the system:

In [5]:
an.graph.show(mdl, gtype='fxnflowgraph')

The functions are:
- `Operator`, the operator of the rover who turns it on/off and takes control if needed (not fully implemented)
- `Power`, the battery and power supply
- `Communications`, the radio used to relay back position/etc. to the operator
- `Avionics`, the computer/control system which determines where to direct the given position information
- `Override`, the system that enables the operator to control the rover instead
- `Perception`, the camera/AI system which percieves the ground and then (not fully implemented)
- `Environment`, which defines the lines and/or changes to the environment based on environmental conditions and/or the movement of the system

The flows are:
- `Control`, the physical interactions between the operator and the rover (i.e. on/off)
- `Comms`, the communications between the operator and the rover (position, video, etc.)
- `OverrideComms`, the communications between the operator and the 
- `Pos_Signal`, the rover's internal idea of its position and velocity
- `EE5`, `EE12`, `EE15`, the electrical power provided by the  battery
- `AvionicsControl`, the contol signals from the avionics to the drive system
- `Video`, the video feed/signal from the camera to the avionics
- `MotorControl`, the control signals to the drive system(s)
- `Ground`, the real position of the rover and the ground/environment/etc.

The model is presently a mixed static/dynamic propagation model in which most function behaviors propagate over time, except for the power system, which propagates in a dynamic step. This may change depending on the necessity to model interactions between these systems. This is shown below.

In [6]:
an.graph.exec_order(mdl, gtype='fxnflowgraph')

The run order for this model is additionally shown below.

In [7]:
an.plot.dyn_order(mdl, rotateticks=True)

## Model simulation

Currently, the model can be simulated in a few situations for lane-tracking: a sine wave and a 90-degree turn.

Below shows the 90-degree turn. As shown, the centerline tracking is not perfect, with a fairly large deviation in distance (current limit for viewing the line in the model is 1 meter). This should be controlled in the future by lowering the timestep (enabling more corrections) or enabling some reduction to velocity when going around corners.

In [8]:
mdl = Rover(params=gen_params('turn'))
endresults, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

In [9]:
mdlhist['time']

Below shows the sine wave. As shown, this curve has a relatively low amplitude (see the y-axis), which results in low tracking error.

In [10]:
mdl = Rover(params=gen_params('sine'))
endresults, mdlhist = prop.nominal(mdl)
plot_map(mdl, mdlhist)

In [11]:
mdlhist['time']

The performance of the rover in these situations is dependent on the parameters of the situation (e.g., the radius of the curve and the amplitude of the sine wave). Thus, it is important to define the operational envelope for the system. This can be done using a `NominalApproach`, which can be used to define ranges of variables to simulate the system under.

In [12]:
from fmdtools.sim.approach import NominalApproach

In this approach we define parameter ranges for the two major situations--a wavelength and amplitude for the sine wave, and a radius and start location for the turn.

In [13]:
app = NominalApproach()
app.add_param_ranges(gen_params,'sine', 'sine', amp=(0, 8, 0.2), wavelength=(10,50,10))
app.add_param_ranges(gen_params,'turn','turn', radius=(5,40,5), start=(0, 20,5))

In [14]:
app

This approach can then be run using `prop.nominal_approach`

In [15]:
endclasses, mdlhists= prop.nominal_approach(mdl, app, pool = mp.Pool(5))

We can then use these results to visualize the operational envelope for the system--which sets of parameters lead to the system getting lost/going off track, and which sets of parameters the system is capable of accomodating.

In [16]:
fig = an.plot.nominal_vals_2d(app, endclasses, 'amp', 'wavelength', nomlabel='True', metric='at_finish')

As shown, if the amplitude of the sine curve is too high at low wavelength, it causes the rover to go off track and not be able to complete the mission.

In [17]:
fig = an.plot.nominal_vals_2d(app, endclasses, 'radius', 'start', nomlabel='True', metric='at_finish')

Similarly, if the radius of the curve is too small, it can additionally lead to the rover going off course by overshooting the curve.

In [18]:
fig = plot_trajectories({k:v for k,v in mdlhists.items() if k[0:4]=='sine'})

In [19]:
fig = plot_trajectories({k:v for k,v in mdlhists.items() if k[0:4]=='turn'})

In [20]:
[s for s,v in app.scenarios.items() if (v.inputparams.get('wavelength','')==40 and v.inputparams.get('amp','')==9)]

In [21]:
mdlhist